In [ ]:
"""

- Imoveis na periferia de São Paulo são subvalorizados?

- A ideia que norteia o projeto é entender se os imoveis que em raio de 30km do marco 0 da cidade de São Paulo são subvalorizados em comparação ao centro ou bairros nobres
- Para possibilitar uma analise de recomendação de investimento nesses imoveis com base em expectativa de lucro, tempo de retorno e analise de risco do mercado.
- Possiveis interessados na analise exploratoria: Startups do mercado imobiliario, Fundos Imobiliarios, Construtoras, Instituições financeiras que fornecem credito imobiliario

- analise baseada nos imoveis disponiveis nos sites Loft, Quinto Andar e Zap Imoveis

- a primeira missão seria conseguir as informações dos imoveis listados com [id_anuncio, endereco, preco, m2, quartos, banheiro, vaga de garagem, tipo imovel, novo/usado, rating atribuido por mim]
- se possivel obter alguma informação sobre a venda desses imoveis (talvez diario oficial para buscar tranferencias de imoveis)
- com base no endereço começamos a classificar se o imovel está em area periferica, neutra ou nobre
- pós classificação começamos a fazer as comparações de rating e preco de imoveis similares em locais diferentes
- tirar mais ideias sobre a situação imobiliaria de sao paulo
- tentar criar algoritmo que busca oportunidades de lucro 

-- linkar com problemas habitacionais da cidade
-- se possivel realizar uma regressao para comparar o quanto cada variavel impacta no preço do m2/total do imovel e nao se esquecer de adicionar uma variavel para segurança do bairro e outros fatores externos


WebScreaping 
Beautiful soup
Seleniun
Paralelização 

--quando fizer o webscraping procurar as <tags> que contenha apenas a info que eu quero
--para buscar diversas tags diferentes é interessante criar uma função com condicionais para cada type de informação dentro da tag

EXEMPLO
tags = ['p','h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'h7']
text_list = []
for element in soup.find_all(tags):
    text_list.append(element.text)
print('\n'.join(text_list))


"""

In [ ]:
"""TASK LIST 
- fazer o scraping de uma das variaveis do imovel em uma pagina simples [feito]

- fazer o scraping de todas as variaveis do imovel
- criar uma spider com esse scraping
- começar a pensar como aplicar isso para a pagina base site


"""

In [1]:
# Imports que fazemos em um webscraping

import tqdm
import pandas as pd
import numpy as np

import requests
from datetime import datetime

from bs4 import BeautifulSoup
from lxml import etree

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.keys import By
# para importar o Xpath e usa-lo com o seleniun 
# Learn xpath https://topswagcode.com/xpath/
# reference https://www.w3schools.com/xml/xpath_syntax.asp



In [11]:
#localização do chromewebdriver
chrome_path = r'C:\Users\T-GAMER\Desktop\Ironhack DA\Week 4\chromedriver_win32\chromedriver'
ser = Service(chrome_path)

In [12]:
#chromewebdriver deve estar aberto para o codigo funcionar
driver = webdriver.Chrome(service=ser)

# Base coleta de dados primeira pagina

In [13]:
url = 'https://loft.com.br/imovel/apartamento-rua-francisco-rossano-vila-prudente-sao-paulo-1-quarto-40m2/ycf79w'

In [14]:
driver.get(url)

In [ ]:
driver.current_url

In [ ]:
#metodo do webdriver 'find_element para procurar um elemento dentro da minha URL pelo xpath' 
#peguei o caminho full_xpath do valor no grande box a direita
driver.find_element('xpath','/html/body/div[1]/section/div[2]/div[2]/div/div/div/div/div/div/ul/li[2]/p').text

In [ ]:
#peguei o caminho full_xpath do ID do Imóvel nas letras cinzas embaixo das fotos
#será necessário limpar o DataFrame para criar uma chave unica deste ID
driver.find_element('xpath', '/html/body/div[1]/section/div[2]/div[1]/div[1]/div/div/div[1]/nav/ol/li[9]/span').text

In [ ]:
#peguei o caminho full_xpath da rua nas letras cinzas embaixo das fotos 
driver.find_element('xpath', '/html/body/div[1]/section/div[2]/div[1]/div[1]/div/div/div[1]/nav/ol/li[7]/a').text

In [ ]:
#peguei o caminho full_xpath do tipo do imovel nas descrições em preto embaixo das fotos 
#será necessário limpar o tipo com regex, seria uma boa ideia entender os tipos de imovel dentro do site e depois filtrar eles
driver.find_element('xpath', '/html/body/div[1]/section/div[2]/div[1]/div[3]/div[2]/div[1]/p').text

In [ ]:
#peguei o caminho full_xpath do titulo nas descrições em preto embaixo das foto para extrair a quantidade de quartos, suites e vagas de gaaragem  
#será necessário limpar o tipo com regex
driver.find_element('xpath','/html/body/div[1]/section/div[2]/div[1]/div[1]/div/div/div[2]/h1').text

In [ ]:
#peguei o caminho full_xpath das descrições para extrair a quantidade de banheiros.  
#será necessário limpar o tipo com regex
driver.find_element('xpath','/html/body/div[1]/section/div[2]/div[1]/div[3]/div[2]/div[5]/p').text

# Teste de Coleta outras paginas (spider)

In [21]:

def coleta_infos():
    
    """Essa funcao coleta os dados de um imovel na pagina de anuncio do site Loft por meio do Xpath e salva estes dados dentro de um dicionario
    Atualmente para a funcao ser executada corretamente é necessario carregar a pagina previamente no navegador de maneira manual
    Eventualmente o site atualiza de um dia para o outro e os caminhos de Xpath devem ser atualizados"""
    
    preco_imov = driver.find_element('xpath','/html/body/div[1]/section/div[2]/div[2]/div/div/div/div/div/div[1]/ul/li[1]/p').text
    id_imov = driver.find_element('xpath', '/html/body/div[1]/section/div[2]/div[1]/div[1]/div/div/div[1]/nav/ol/li[9]/span').text
    endereco = driver.find_element('xpath', '/html/body/div[1]/section/div[2]/div[1]/div[1]/div/div/div[1]/nav/ol/li[7]/a').text
    titulo_imov = driver.find_element('xpath','/html/body/div[1]/section/div[2]/div[1]/div[1]/div/div/div[2]/h1').text
    banheiro = driver.find_element('xpath','/html/body/div[1]/section/div[2]/div[1]/div[3]/div[3]/div[5]/p').text
    
    global df_teste
    df_teste = {'Preco': preco_imov, 'Id': id_imov, 'Endereco': endereco, 'Descricoes Imovel': titulo_imov, 'Banheiros': banheiro}


In [22]:
#com a URL fora da função o codigo funciona para captar as informações de paginas unicas, algo como atualizar a url manualmente a cada coleta

url = driver.current_url
driver.get(url)
coleta_infos()

In [23]:
df_teste

{'Preco': 'R$ 299.000',
 'Id': 'Imóvel YCF79W',
 'Endereco': 'Rua Francisco Rossano',
 'Descricoes Imovel': 'Apartamento à venda em Vila Prudente com 40 m², 1 quarto, 1 suíte, 1 vaga',
 'Banheiros': '1 Banheiro'}

In [24]:
df = pd.DataFrame(df_teste,index = [0])

In [25]:
df

,Preco,Id,Endereco,Descricoes Imovel,Banheiros
0,R$ 299.000,Imóvel YCF79W,Rua Francisco Rossano,Apartamento à venda em Vila Prudente com 40 m²...,1 Banheiro


In [ ]:
# Começar automatização do codigo para rodar em varias paginas.

In [28]:
url_compra = 'https://loft.com.br/venda/imoveis/sp/sao-paulo'
driver.get(url_compra)


In [36]:
# Meu problema atual é que o programa clica no imovel mas não se altera a current URL. 
# Preciso que ele realize o clique, colete as informações e feche a aba para clicar no proximo link

link_imov = driver.find_element('xpath', '/html/body/div[1]/section/div[4]/div[1]/div/div[1]/div[3]/a')
link_imov2 = driver.find_element('xpath', '/html/body/div[1]/section/div[4]/div[1]/div/div[1]/div[4]/a')
# Quando trabalhar na automatização do clique note que apenas o ultimo Div muda o argumento de posição, seria interessante 
# criar uma extrutura de loop alterando esta posição para pegar todos os links

link_imov.click()
driver.current_url


'https://loft.com.br/venda/imoveis/sp/sao-paulo'